# Import Packages

In [1]:
import tensorflow as tf
import tensorflow.keras as keras

print(tf.__version__)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [2]:
from keras.applications.vgg19 import VGG19
from keras.models import Model,Sequential
from keras.layers import Input, LSTM, Multiply, Dense, Embedding, Flatten, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, Adam


Using TensorFlow backend.


In [3]:
import json
import pandas as pd
import numpy as np
import glob
import os
import spacy
import h5py
import pickle

In [7]:
%cd /home/anthony/Projects/42028-assignment3/
#%cd /content/gdrive/My Drive/42028-DL-CNN/Assignment3/
%ls

/home/anthony/Projects/42028-assignment3
Annotations_Train_mscoco.zip  Pipfile.lock                train2014.zip
Annotations_Val_mscoco.zip    predict/                    train_set_index.csv
app.py                        __pycache__/                uploads/
data/                         Questions_Train_mscoco.zip  val2014.zip
models/                       Questions_Val_mscoco.zip    val_set_index.csv
model_training/               README.md                   VQA/
notebooks/                    settings.py
Pipfile                       templates/


# Prepare Training and Validation Dataframes

In [4]:
def list_images_from_dir(path):
  #img_list = glob.glob(path + '/*.jpg')
  img_list = os.listdir(path)
  img_list = [img.replace(path, '') for img in img_list]
  print(len(img_list))
  return img_list

def remove_missing_images(df, col_name, img_list):
  return df[df[col_name].isin(img_list)]

## Prepare train

In [0]:
#with open('data/coco/annotations/instances_train2014.json') as json_file:  
with open('data/coco/annotations/annotations/instances_train2014.json') as json_file:  
    train_data = json.load(json_file)

In [0]:
images_df = pd.DataFrame(train_data['images'])
annotations_df = pd.DataFrame(train_data['annotations'])
categories_df = pd.DataFrame(train_data['categories'])
images_df = images_df[['file_name','height','id','width']]
annotations_df = annotations_df[['category_id','image_id']]
categories_df = categories_df[['id', 'name']]

train_set = annotations_df.merge(images_df, how='left', left_on='image_id', right_on='id').drop('id', axis=1)
train_set = train_set.merge(categories_df, how='left', left_on='category_id', right_on='id').drop('id', axis=1)

In [0]:
train_dir = "data/coco/images/train2014/"
train_img_list_file = 'data/coco/images/train2014_img_list.pkl'

In [0]:
with open (train_img_list_file, 'rb') as fp:
    train_images = pickle.load(fp)
len(train_images)

In [0]:
train_set = remove_missing_images(train_set, 'file_name', train_images)

train_set.shape, train_set['name'].nunique()

# Train Image Model

In [0]:
%mkdir models
%mkdir models/vgg19

mkdir: cannot create directory ‘models’: File exists
mkdir: cannot create directory ‘models/vgg19’: File exists


In [0]:
# Updated to do image augmentation
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_dataframe(
    dataframe=train_set, 
    directory=train_dir, 
    x_col="file_name", 
    y_col="name", 
    class_mode="categorical", 
    target_size=(150, 150), 
    batch_size=32)


# Update model to extract Feature Maps Layer

In [0]:
model_name = f'models/vgg19/vgg19_feature_extractor.h5'

In [0]:
model_feat_extract = keras.models.load_model(model_name)

# Extract Feature Maps

In [0]:
step_size_train=train_generator.n//10
train_preds = model_feat_extract.predict_generator(train_generator, max_queue_size=10, workers=-1, use_multiprocessing=True, verbose=1, steps=step_size_train)

with h5py.File('vgg19_img_features_train.h5','w') as hf:
  hf.create_dataset('train', data=train_preds)

In [0]:
with h5py.File(img_feature_file,'r') as hf:
  train_img_feats =  np.array(hf.get('train'))
  val_img_feats =  np.array(hf.get('val'))